In [10]:
import pandas as pd
from custom_functions_scRBA import *

In [11]:
#### LOAD INPUTS AND PARAMETERS
# Stoichiometry
df_stoich = pd.read_excel('./model/RBA_stoichiometry.xlsx')
df_stoich.index = df_stoich.id.to_list()

# Enzymes' apparent turnover rates (k_app) (unit: 1/s)
# (script will convert the unit to 1/h)
df_kapp = pd.read_excel('./input/PARAMS_kapp_compiled.xlsx')
df_kapp.index = df_kapp.id.to_list()

# Load protein sequence lengths
df_pro = pd.read_excel('./input/PROTEIN_stoich_curation.xlsx')
df_pro.index = df_pro.id.to_list()

# Ribosome efficiency (amino acids translated per active ribosome per second)
# (script will convert the unit to per hour)
kribo = 10.5

# Dummy protein's length (assigned to be the median protein length)
NAA_dummy = 401

In [12]:
eqn_list = []

In [13]:
### Ribosome synthesis requirement coupled to protein translation
# Mitochondrial
idx = df_stoich[(df_stoich.coupling_type == 'prot_ribo') & \
                (df_stoich.coupling_species == 'ribomito')].index
lhs = "v('RIBOSYN-ribomito') * %kribomito%"
rhs = "%mu% * sum(j$mito_translation(j), NAA(j) * v(j))"
eqn_list.append(lhs + ' =e= ' + rhs + ';')

rxn_list = ["'" + i + "'" for i in idx if i != '']
rxn_list = ['/'] + rxn_list + ['/']
with open('./model/RBA_mito_translation.txt', 'w') as f:
    f.write('\n'.join(rxn_list))

In [14]:
# Nucleus
idx = df_stoich[(df_stoich.coupling_type == 'prot_ribo') & \
                (df_stoich.coupling_species == 'ribonuc')].index
lhs = "v('RIBOSYN-ribonuc') * %kribonuc%"
rhs = "%mu% * sum(j$nuc_translation(j), NAA(j) * v(j))"
eqn_list.append(lhs + ' =e= ' + rhs + ';')

rxn_list = ["'" + i + "'" for i in idx if i != '']
rxn_list = ['/'] + rxn_list + ['/']
with open('./model/RBA_nuc_translation.txt', 'w') as f:
    f.write('\n'.join(rxn_list))

In [15]:
### Enzyme synthesis requirement coupled to metabolic reaction rate
idx = df_stoich[df_stoich.coupling_type == 'rxn_enz'].index
for i in idx:
    lhs = "v('ENZSYN-" + df_stoich.id[i][4:] + "') * " + \
        str(round(df_kapp.loc[i, 'kapp (1/s)']*3600,6))
    rhs = "%mu% * v('" + i + "')"
    eqn_list.append(lhs + ' =e= ' + rhs + ';')
    
eqn_idx = ['eqn'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['eqn'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]

with open('./model/RBA_capacityConstraints_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))
with open('./model/RBA_capacityConstraints_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))

In [16]:
### Write protein length file (NAA)
pro_list = []
for i in df_pro.index:
    pro_list.append("'PROSYN-" + df_pro.id[i] + "' " + str(len(df_pro.sequence[i]) - 1))
pro_list.append("'PROSYN-PROTDUMMY' " + str(NAA_dummy))

pro_list = ['/'] + pro_list + ['/']
with open('./model/RBA_proteinLength.txt', 'w') as f:
    f.write('\n'.join(pro_list))

In [17]:
### Write prosyn reaction
idx = [i for i in df_stoich.index if i[:7] == 'PROSYN-']
prosyn = ["'" + i + "'" for i in idx]
prosyn = ['/'] + prosyn + ['/']
with open('./model/RBA_rxns_prosyn.txt', 'w') as f:
    f.write('\n'.join(prosyn))

In [18]:
eqn_list

["eqn0.. v('RIBOSYN-ribomito') * %kribomito% =e= %mu% * sum(j$mito_translation(j), NAA(j) * v(j));",
 "eqn1.. v('RIBOSYN-ribonuc') * %kribonuc% =e= %mu% * sum(j$nuc_translation(j), NAA(j) * v(j));",
 "eqn2.. v('ENZSYN-BTDDH_c_FWD-YAL060W') * 5880000.0 =e= %mu% * v('RXN-BTDDH_c_FWD-YAL060W');",
 "eqn3.. v('ENZSYN-BTDDH_c_REV-YAL060W') * 5880000.0 =e= %mu% * v('RXN-BTDDH_c_REV-YAL060W');",
 "eqn4.. v('ENZSYN-13GS_c_FWD-GSC2RHO1') * 48000.768012 =e= %mu% * v('RXN-13GS_c_FWD-GSC2RHO1');",
 "eqn5.. v('ENZSYN-13GS_c_FWD-GSC1RHO1') * 48000.768012 =e= %mu% * v('RXN-13GS_c_FWD-GSC1RHO1');",
 "eqn6.. v('ENZSYN-16GS_c_FWD-YPR159W') * 255240.0 =e= %mu% * v('RXN-16GS_c_FWD-YPR159W');",
 "eqn7.. v('ENZSYN-16GS_c_FWD-YGR143W') * 255240.0 =e= %mu% * v('RXN-16GS_c_FWD-YGR143W');",
 "eqn8.. v('ENZSYN-PRMICI_c_FWD-YIL020C') * 115199.410179 =e= %mu% * v('RXN-PRMICI_c_FWD-YIL020C');",
 "eqn9.. v('ENZSYN-P5CD_m_FWD-YHR037W') * 5400.0 =e= %mu% * v('RXN-P5CD_m_FWD-YHR037W');",
 "eqn10.. v('ENZSYN-P5CD_m_REV-Y